### CS 524 &mdash; Introduction to Optimization &mdash; Spring 2022 ###

# Orientation Planner for UW-Madison Students

####  Grant Kirner, Heather Jia,  Sabeeka Khan




## 1. Introduction ##

Orientation for colleges is a first step for many students to get accustomed to their campus and peers. There is a lot of planning that goes behind a multi-day orientation such as food, events, and number of workers needed. To assist in planning an orientation at UW-Madison, our optimization problem will try to propose a schedule over a two-day weekend to maximize prospective student satisfaction, while minimizing the total number of workers needed for the orientation.  

We measure the cost of an event by the number of workers needed times payrate for the event. The format of the event files and the rating survey is descibed in this report. The organizer can also change the weight of the total satisfaction and the cost of the trip. By running our code, the organizer will get a schedule of the orientation including what event to do at each hour and a summary of the cost. We will analyze the tradeoffs between satisfaction of students and total cost of the orientation. This will involve changing the objective function or implementing a lambda value with the combined objective function.

<p>
    <center><img src="https://i.imgur.com/nktMcmQ.jpeg" width=400></center>
    <center><em>UW-Madison Student Orientation</em></center>
</p>

## 2. Mathematical Model ##

In this section, we will discuss how to transfer the constraints and objectives into mathematical form. For the sake of convenience, we define $\mathcal{I}$ as the set containing all hours in this weekend, $\mathcal{J}$ as the set of all events selected for the orientation, and $\mathcal{K}$ as the set of all students attending the orientation weekend. 

### 2.1. Modeling Assumptions

- The decision variables will account for each of the 20 hours in the 2-day weekend. 
- The model will produce event arrangments from 8am to 6pm Saturday through Sunday including lunches.
- Each hour will be a plan for what the students should do in that given hour. 
- We measure the cost of the events by the number of workers needed times pay rate instead of a monetary value.
- Each event has a corresponding number of workers needed

### 2.2. Parameters

- The each student's satisfaction of each event is obtained from a survey collected from all students in the group. $s_{k,j}$ is the satisfaction group member $k$ gets from doing activity $j$. The higher the number is, the more the students will like the event.
- The number of workers required for each activity is collected in a list $W$, where $W_j$ is the number of workers required for event $j$
- Total number of students in the group is $num\_s$. By default, $num\_s=10$. This parameter can be manually changed by the orientation organizer.
- The papameter $total\_workers$ is the maximum number of workers that are available for the orientation. By default, $total\_workers=200$.
- The minimum satisfaction for each student, $S_{min}$, can be manually set by the organizer of the orientation weekend. 
- Specific times that each event can be held at is stored in $O$, where $O_{i,j}$ is 0 when activity $j$ is available to be held at hour $i$, and 1 otherwise.
- Let $S$ be a vector where $S_i$ is 1 when the students are free and no events are organized at hour $i$, and 0 otherwise.
- Let $H\_time$ be the total number of hours the students are free and no events are organized.
- Let $L$ be a vector where $L_i$ is 1 when hour $i$ is when they will eat lunch and 0 otherwise. These times are 12pm and 1pm Saturday and Sunday, can be manually set by the organizer.
- Let $F$ be the set of places to get food. The data can be found in the data collection section.
- Let $NumberLunches$ be the total number of lunches in the weekend trip. By default, $NumberLunches=2$.


### 2.3. Decision Variables

- The decision variable will be a matrix $X$ where its element $X_{i,j}$ represents whether or not event j will be held at hour i. 
- Another decision variable wil be $Z$, where $Z_j$ is 1 if event j is held sometime during the orientation weekend, and 0 otherwise.

### 2.4. Constraints

#### 2.4.1. Total Cost Constraint

The total number of workers required can not be larger than maximum number of workers that are available for the orientation. 
$$
 0 \leq \quad \sum_{j=1}^{\Vert \mathcal{J} \Vert}z_jW_{j}\leq total\_workers
$$

<a id='constraints'></a>
#### 2.4.2. Minimum Satisfaction Constraint

The amount of total satisfaction can’t be lower than a specified amount for each student.
    
$$\forall k \in \mathcal{K}, \quad \sum_{i=1}^{\Vert \mathcal{I} \Vert}\sum_{j=1}^{\Vert \mathcal{J} \Vert}s_{i,j}S_{i,j}\geq S_{min}$$



#### 2.4.3. Lunch Time Constraint
The group must eat lunch each day,
$$\forall k \in \mathcal{K}, \quad \sum_{i=1}^{\Vert \mathcal{I} \Vert}\sum_{f=1}^{\Vert \mathcal{F} \Vert}L_{i}X_{i,j} = TimeAtLunch*NumberLunches$$

#### 2.4.4. Rest Time Constraint

The students will be free during this time. 
$$
\forall j \in H, \quad \sum_{i=1}^{\Vert \mathcal{I} \Vert}S_{i}X_{i,j}=H\_time
$$

<a id='constraints'></a>
#### 2.4.5. Event Availability Constraint

The students can only do an event if that event is available at a specific time.
    
$$
\quad \sum_{i=1}^{\Vert \mathcal{I} \Vert}O_{i,j}X_{i,j}=0
$$

<a id='constraints'></a>
#### 2.4.6. No Repetition in Event Arrangement

The students will not have an event more than once in the weekend.

$$
\forall \; 1 < i \leq \Vert \mathcal{I}\Vert \; j \notin H, \lambda_{i,j}=|X_{i,j} - X_{i-1,j}|\\
\forall j \notin H, \lambda_{1,j}=X_{1,j}\\
\forall j \in \mathcal{J}, \quad \sum_{i=1}^{\Vert \mathcal{I} \Vert}\lambda_{i,j}<=2
$$

<a id='constraints'></a>
#### 2.4.7. One Event at A Time

The students is either doing an event at a certain time or they are not,

$$
\forall i \in \mathcal{I}, \quad \sum_{j=1}^{\Vert \mathcal{J} \Vert} X_{i,j}=1
$$

<a id='constraints'></a>
#### 2.4.8. Introduce z

If z_j is 1, then the students did event i, where M is the upper bound.

$$
z_{j}\in \{0,1\}\\
\forall j, \quad \sum_{i=1}^{\Vert \mathcal{I} \Vert}X_{i,j}\leq Mz_j
$$

### 2.5. Objective Function

#### 2.5.1. Student Satisfaction

Maximizing the total amount of “satisfaction” of the group,
$$\max \quad \sum_{k=1}^{\Vert \mathcal{K} \Vert}\sum_{i=1}^{\Vert \mathcal{I} \Vert}\sum_{j=1}^{\Vert \mathcal{J} \Vert}s_{i,j}X_{i,j} $$

#### 2.5.2. Total Cost

Minimizing the total planning cost 

$$\min \sum_{j=1}^{\Vert \mathcal{J} \Vert}z_{j}C_{j} $$

#### 2.5.3. Combing both costs


$$\max \quad \sum_{k=1}^{\Vert \mathcal{K} \Vert}\sum_{i=1}^{\Vert \mathcal{I} \Vert}\sum_{j=1}^{\Vert \mathcal{J} \Vert}s_{i,j}X_{i,j}  -  \lambda\sum_{j=1}^{\Vert \mathcal{J} \Vert}z_{j}C_{j} $$

## 3. Julia Implementation

In [10]:
using JuMP
using Cbc
using CSV
using DataFrames

In [40]:
function read_csv(filename)
    raw = CSV.read(filename, DataFrame)
    (rows, cols) = size(raw)
    raw = raw[1:rows, 2:cols]
    data = Matrix(raw)
    #data = convert(Matrix{Float64}, raw)
    return data
end

budget = 500000
num_students = 2
min_satisfaction = 1
cost_per_worker = 0.5


survey_data = read_csv("survey_data.csv")
hours_open = read_csv("hours_open.csv") # 0 is open. 1 is closed.
time_limits = read_csv("time_limits.csv") # duration of each event. #hours
workers_needed = read_csv("workers_needed.csv")
workers_available = read_csv("workers_available.csv")
events = Matrix(CSV.read("events.csv",DataFrame, header=0))
events = rstrip.(events)
times = Matrix(CSV.read("times.csv",DataFrame,delim=",", header=0))
times = rstrip.(times)


# manditory_lunch_time = [5; 29]
# lunch_indices = [18; 19; 20] # lunch indices in events.csv
(num_hours, num_events) = size(hours_open)

In [41]:
m = Model(with_optimizer(Cbc.Optimizer, logLevel=0))


@variable(m, x[1:num_hours,1:num_events], Bin)
@variable(m, lambda[1:num_hours,1:num_events] >= 0)
@variable(m, z[1:num_events], Bin)


# # The group is either doing an event at a certain time or they're not
@constraint(m, [i=1:num_hours, j=1:num_events], x[i,j] <= 1)

# z[j] is 1 if the group did an event j during their orientation
@constraint(m, [j=1:num_events], sum(x[i,j] for i in 1:num_hours) <= num_hours*z[j])

# The group can only do one event at a time
@constraint(m, [i=1:num_hours], sum(x[i,j] for j in 1:num_events) == 1)

# The group cannot do an event for an unlimited amount of time
# @constraint(m, [j=1:num_events], sum(x[i,j] for i in 1:num_hours) == time_limits[j])

# The total cost of the orientation week is less than the total budget
# @constraint(m,  sum(z[j]*cost_per_worker*workers_needed[j] for j in 1:num_events) <= budget)

# Each person in the group must get some baseline of sastisfaction
@constraint(m, [p=1:num_students], sum(sum(survey_data[p,j]*x[i,j] for i in 1:num_hours) for j in 1:num_events) >= min_satisfaction)

# The group can only do an event if it is open
@constraint(m, [j=1:num_events], sum(hours_open[i,j]*x[i,j] for i in 1:num_hours) == 0)



# The group can only do an event once
# lambda 1 one when the group starts or stops an event
# for j in 1:num_events

#     for i in 2:num_hours
#         @constraint(m, lambda[i,j] >= x[i,j] - x[i-1,j])
#         @constraint(m, lambda[i,j] >= -x[i,j] + x[i-1,j])
#     end
# end
# @constraint(m, [j=1:num_events], sum(lambda[i,j] for i in 1:num_hours) <= 2)

# Our objective is to maximize satisfaction!
@objective(m, Max, sum(sum(sum(survey_data[p,j]*x[i,j] for p in 1:num_students) for i in 1:num_hours) for j in 1:num_events))
optimize!(m)

println(termination_status(m))
x_values = value.(x)

for i in 1:num_hours
    print(times[i], ":\t")
    for j in 1:num_events
        if round(x_values[i,j]) == 1
            println(events[j])
            break
        end
    end
end

satisfaction = sum(sum(sum(survey_data[p,j]*value(x[i,j]) for p in 1:num_students) for i in 1:num_hours) for j in 1:num_events)
#cost = sum(value(z[j]*cost_per_worker*workers_needed[j] for j in 1:num_events))
# println()
# println("Total enjoyment: ", satisfaction)
# println("Total cost: ",cost, " dollars")
    


OPTIMAL
8 AM Saturday:	Current Student Panel
9 AM Saturday:	Current Student Panel
10 AM Saturday:	Academic Advising
11 AM Saturday:	Academic Advising
12 PM Saturday:	Toppers Lunch
1 PM Saturday:	Academic Advising
2 PM Saturday:	Academic Advising
3 PM Saturday:	Academic Advising
4 PM Saturday:	Current Student Panel
5 PM Saturday:	Current Student Panel
6 PM Saturday:	Current Student Panel
8 AM Sunday:	Current Student Panel
9 AM Sunday:	Current Student Panel
10 AM Sunday:	Academic Advising
11 AM Sunday:	Academic Advising
12 PM Sunday:	Toppers Lunch
1 PM Sunday:	Academic Advising
2 PM Sunday:	Academic Advising
3 PM Sunday:	Academic Advising
4 PM Sunday:	Current Student Panel
5 PM Sunday:	Current Student Panel
6 PM Sunday:	Current Student Panel


352.0

## 4. Preliminary Analysis and next steps



From our current results, we see that the optimizer wants to hold the same event over and over again, so we need to do some debugging to ensure each event can be held a maximum of 1 time.This result is also only optimizing satisfaction, so we still need to run the simulations for optimizing cost and a combination of both. One thing that we see being the toughest part for our eventual optimal solution is providing a constraint to make sure that there is enough time available over a time period to hold events that last more than 1 hour.